In [2]:
from importlib import reload
from load_cluster_data import load_cluster_data
import gc

import numpy as np
import torch
import pandas as pd 
import seaborn as sns
import collections

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

float_type = { 
    "device" : device, 
    "dtype" : torch.float, # save memory
}

hypers = {
    "eta" : 1., 
    "alpha_prior" : 1., # karin had 0.65 
    "pi_prior" : 1.
}

K = 10

import plotnine as p9
import scipy.sparse as sp
import matplotlib.pyplot as plt 
import splicing_PCA_utils
from nuclear_norm_PCA import sparse_sum

from pca_kmeans_init import pca_kmeans_init
from betabinomo_LDA_singlecells_kinit import *
import betabinomo_LDA_singlecells_kinit
reload(betabinomo_LDA_singlecells_kinit)


cpu


/gpfs/commons/home/kisaev/.local/lib/python3.9/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.


<module 'betabinomo_LDA_singlecells_kinit' from '/gpfs/commons/home/kisaev/leafcutter-sc/src/beta-binomial-lda/betabinomo_LDA_singlecells_kinit.py'>

### Load data

In [6]:
# different version of load_cluster_data here to match the dataset 
# load data 

input_file = '/gpfs/commons/groups/knowles_lab/Karin/data/GTEx/GTEx_junction_cluster_counts.h5'
summarized_data = pd.read_hdf(input_file, 'df')
summarized_data.head()

,SAMPID,Cluster,Cluster_Counts,Name,Description,junc_count,SMTS,SMTSD,Strand,gene_id,gene_name,Count,junc_ratio,sample_id_index,junction_id_index
0,GTEX-1117F-0226-SM-5GZZ7,3616.0,8634,chr1_44777790_44777999,ENSG00000142937.11,8634,Adipose Tissue,Adipose - Subcutaneous,+,ENSG00000142937,RPS8,1.0,1.0,0,13
1,GTEX-1117F-0226-SM-5GZZ7,10809.0,20,chr1_209789786_209790494,ENSG00000117595.10,20,Adipose Tissue,Adipose - Subcutaneous,-,ENSG00000117595,IRF6,1.0,1.0,0,12
2,GTEX-1117F-0226-SM-5GZZ7,17722.0,46,chr2_233467204_233468422,ENSG00000077044.10,46,Adipose Tissue,Adipose - Subcutaneous,+,ENSG00000077044,DGKD,1.0,1.0,0,17
3,GTEX-1117F-0226-SM-5GZZ7,24002.0,15,chr2_53948538_53949142,ENSG00000068878.14,15,Adipose Tissue,Adipose - Subcutaneous,-,ENSG00000068878,PSME4,1.0,1.0,0,18
4,GTEX-1117F-0226-SM-5GZZ7,31293.0,60,chr3_179328352_179329617,ENSG00000121864.9,60,Adipose Tissue,Adipose - Subcutaneous,+,ENSG00000121864,ZNF639,1.0,1.0,0,19


In [7]:
print(summarized_data.SMTS.unique())

['Adipose Tissue' 'Muscle' 'Blood Vessel' 'Heart' 'Uterus' 'Vagina'
 'Breast' 'Skin' 'Salivary Gland' 'Brain' 'Adrenal Gland' 'Thyroid' 'Lung'
 'Spleen' 'Pancreas' 'Esophagus' 'Stomach' 'Colon' 'Small Intestine'
 'Prostate' 'Testis' 'Nerve' 'Blood' 'Pituitary' 'Ovary' 'Liver' 'Kidney'
 'Cervix Uteri' 'Fallopian Tube' 'Bladder']


In [8]:
summarized_data['sample_id_index'] = summarized_data.groupby('SAMPID').ngroup()
summarized_data['junction_id_index'] = summarized_data.groupby('Name').ngroup()
coo = summarized_data[["sample_id_index", "junction_id_index", "junc_count", "Cluster_Counts", "Cluster", "junc_ratio"]]

In [10]:
# just some sanity checks to make sure indices are in order 
cell_ids_conversion = summarized_data[["sample_id_index", "SAMPID", "SMTS", "SMTSD"]].drop_duplicates()
cell_ids_conversion = cell_ids_conversion.sort_values("sample_id_index")

junction_ids_conversion = summarized_data[["junction_id_index", "Name", "Cluster", "gene_id", "gene_name"]].drop_duplicates()
junction_ids_conversion = junction_ids_conversion.sort_values("junction_id_index")

In [11]:
# make coo_matrix for junction counts
coo_counts_sparse = coo_matrix((coo.junc_count, (coo.sample_id_index, coo.junction_id_index)), shape=(len(coo.sample_id_index.unique()), len(coo.junction_id_index.unique())))
coo_counts_sparse = coo_counts_sparse.tocsr()

In [14]:
summarized_data.Name.unique()

array(['chr1_44777790_44777999', 'chr1_209789786_209790494',
       'chr2_233467204_233468422', 'chr2_53948538_53949142',
       'chr3_179328352_179329617', 'chr4_82875823_82878729',
       'chr6_43215327_43216157', 'chr7_100374434_100376120',
       'chr7_1980542_2002064', 'chr8_144436921_144437026',
       'chr9_127808312_127808557', 'chr10_73427769_73428063',
       'chr11_10001023_10002556', 'chr13_45970466_45975282',
       'chr15_43844055_43851086', 'chr16_57070619_57074599',
       'chr17_50550116_50550229', 'chr17_10545503_10547474',
       'chr18_31518275_31519802', 'chr19_14409185_14409302',
       'chr22_29354872_29356425', 'chr2_151568418_151568617',
       'chr11_73390922_73393836', 'chr8_27322500_27322666',
       'chr5_151517766_151521275', 'chr3_55683860_55699377',
       'chr16_737330_739539', 'chr20_46056565_46056896',
       'chr1_148573339_148574052'], dtype=object)

In [25]:
coo_counts_sparse.toarray()[17381,15]

88

In [22]:
summarized_data.tail()

,SAMPID,Cluster,Cluster_Counts,Name,Description,junc_count,SMTS,SMTSD,Strand,gene_id,gene_name,Count,junc_ratio,sample_id_index,junction_id_index
368295,GTEX-ZZPU-2726-SM-5NQ8O,124484.0,42,chr16_57070619_57074599,ENSG00000140853.15,42,Adipose Tissue,Adipose - Subcutaneous,+,ENSG00000140853,NLRC5,1.0,1.0,17381,5
368296,GTEX-ZZPU-2726-SM-5NQ8O,130209.0,5,chr17_50550116_50550229,ENSG00000006282.20,5,Adipose Tissue,Adipose - Subcutaneous,+,ENSG00000006282,SPATA20,2.0,1.0,17381,8
368297,GTEX-ZZPU-2726-SM-5NQ8O,139802.0,42,chr18_31518275_31519802,ENSG00000046604.12,42,Adipose Tissue,Adipose - Subcutaneous,+,ENSG00000046604,DSG2,1.0,1.0,17381,9
368298,GTEX-ZZPU-2726-SM-5NQ8O,148591.0,162,chr19_14409185_14409302,ENSG00000123136.14,162,Adipose Tissue,Adipose - Subcutaneous,-,ENSG00000123136,DDX39A,1.0,1.0,17381,10
368299,GTEX-ZZPU-2726-SM-5NQ8O,159663.0,88,chr22_29354872_29356425,ENSG00000100280.16,88,Adipose Tissue,Adipose - Subcutaneous,-,ENSG00000100280,AP1B1,1.0,1.0,17381,15


In [ ]:
juncs_nonzero = pd.DataFrame(np.transpose(coo_counts_sparse.nonzero()))
juncs_nonzero.columns = ["cell_id_index", "junction_id_index"]
juncs_nonzero["junc_count"] = coo_counts_sparse.data
# do the same for cluster counts 
cells_only = coo[["cell_id_index", "Cluster", "Cluster_Counts"]].drop_duplicates()
merged_df = pd.merge(cells_only, junction_ids_conversion)
coo_cluster_sparse = coo_matrix((merged_df.Cluster_Counts, (merged_df.cell_id_index, merged_df.junction_id_index)), shape=(len(merged_df.cell_id_index.unique()), len(merged_df.junction_id_index.unique())))
coo_cluster_sparse = coo_cluster_sparse.tocsr()
cluster_nonzero = pd.DataFrame(np.transpose(coo_cluster_sparse.nonzero()))
cluster_nonzero.columns = ["cell_id_index", "junction_id_index"]
cluster_nonzero["cluster_count"] = coo_cluster_sparse.data
final_data = pd.merge(juncs_nonzero, cluster_nonzero, how='outer').fillna(0)
final_data["clustminjunc"] = final_data["cluster_count"] - final_data["junc_count"]
final_data["juncratio"] = final_data["junc_count"] / final_data["cluster_count"] 
final_data = final_data.merge(cell_ids_conversion, on="cell_id_index", how="left")

In [ ]:
final_data, coo_counts_sparse, coo_cluster_sparse, cell_ids_conversion, junction_ids_conversion = load_cluster_data(
    input_file) # , celltypes = ["B", "MemoryCD4T"])

# still do preprocessing in scipy
indices = (final_data.cell_id_index, final_data.junction_id_index)
indices_np = np.stack(indices)
junc_counts = sp.coo_matrix((final_data.junc_count, indices))
cluster_counts = sp.coo_matrix((final_data.cluster_count, indices))

In [ ]:
junc_counts_sub, cluster_counts_sub, to_keep = splicing_PCA_utils.filter_junctions(
    junc_counts, cluster_counts, plot = True)

junc_ind_keep = junction_ids_conversion["junction_id_index"][to_keep]
final_data = final_data[final_data.junction_id_index.isin(junc_ind_keep)]

In [ ]:
junction_ids_conversion=junction_ids_conversion[to_keep]
junction_ids_conversion["new_junction_id_index"] = np.arange(junction_ids_conversion.shape[0])

In [ ]:
# re-order the remaining junctions 
final_data = final_data.merge(junction_ids_conversion, on = "junction_id_index")
final_data

In [ ]:
print(junc_counts_sub.toarray()[3, 720]) # should be 3 sanity check of data 
print(cluster_counts_sub.toarray()[3, 720]) # should be 5 sanity check of data 

### Simulate Data

In [ ]:
sim_junc_counts = splicing_PCA_utils.simulate_junc_counts(
    cluster_counts_sub,
    cell_ids_conversion.cell_type.astype('category')
)

In [ ]:
# make dataframe using the following columsn 
sim_junc_counts_flat = pd.DataFrame({"cell_id_index": sim_junc_counts.row, "new_junction_id_index": sim_junc_counts.col, "new_junc_count": sim_junc_counts.data})

# update junction counts in final_data object to be the simulated counts 
final_data = final_data.merge(sim_junc_counts_flat, on = ["cell_id_index", "new_junction_id_index"])

### Prep simulated data as input into LDA 

In [ ]:
sim_data = final_data.copy() # make a copy of the data
sim_data.drop(columns = ["junc_count", "junction_id_index"], inplace = True) # drop the old junction counts and junction id index
# rename columns new_junction_id_index and new_junc_count to junction_id_index and junc_count
sim_data.rename(columns = {"new_junction_id_index": "junction_id_index", "new_junc_count": "junc_count"}, inplace = True)

In [ ]:
my_data = make_torch_data(sim_data, **float_type)

In [ ]:
my_data

### Run LDA

In [ ]:
num_trials = 1 # can't currently run more than 1 or overflow GPU memory :( 
num_iters = 300 # should also be an argument that gets fed in

# loop over the number of trials (for now just testing using one trial but in general need to evaluate how performance is affected by number of trials)
for t in range(num_trials):

    # run coordinate ascent VI
    print(K)

    ALPHA_f, PI_f, GAMMA_f, PHI_f, elbos_all = calculate_CAVI(K, my_data, float_type, hypers = hypers, num_iterations = num_iters)
    elbos_all = np.array(elbos_all)
    juncs_probs = ALPHA_f / (ALPHA_f+PI_f)
    
    theta_f = GAMMA_f / GAMMA_f.sum(1,keepdim=True)
    theta_f_plot = pd.DataFrame(theta_f.cpu())
    theta_f_plot['cell_id'] = cell_ids_conversion["cell_type"].to_numpy()
    theta_f_plot_summ = theta_f_plot.groupby('cell_id').mean()
    print(theta_f_plot_summ)
    
    # plot ELBO
    plt.plot(elbos_all[2:]); plt.show()

In [ ]:
x = theta_f.cpu().numpy()
x -= x.mean(1,keepdims=True)
x /= x.std(1,keepdims=True)
plt.hist(x.flatten(),100)
pd.crosstab( cell_ids_conversion["cell_type"], x.argmax(axis=1) )

In [ ]:
x

In [ ]:
x.shape

In [ ]:
labels=cell_ids_conversion["cell_type"].to_numpy()

In [ ]:
xdf=pd.DataFrame(x)

In [ ]:
from sklearn.decomposition import PCA
pca_dat = PCA(n_components=10)
principalComponents_breast = pca_dat.fit_transform(xdf)

In [ ]:
principal_x_Df = pd.DataFrame(data = principalComponents_breast
             , columns = ['pc1', 'pc2', 'pc3', 'pc4', 'pc5', 'pc6', 'pc7', 'pc8', 'pc9', 'pc10'])
print('Explained variation per principal component: {}'.format(pca_dat.explained_variance_ratio_))


In [ ]:
principal_x_Df['cell_type'] = labels
principal_x_Df

In [ ]:
sns.scatterplot("pc1", "pc2", data=principal_x_Df, hue = "cell_type")

In [ ]:
sns.scatterplot("pc2", "pc3", data=principal_x_Df, hue = "cell_type")

In [ ]:
sns.scatterplot("pc1", "pc3", data=principal_x_Df, hue = "cell_type")